In [ ]:
import mysql.connector

In [ ]:
cnx = mysql.connector.connect(user='webauthor', password='xxxxxx',
                              host='public.ofm-research.org',
                              database='berman')

Objective-C header and class generation code

In [ ]:
query = ("SELECT p.name, p.formula, r.h, r.s, r.v, Cp.k0, Cp.k1, Cp.k2, Cp.k3, Vtp.v1, Vtp.v2, Vtp.v3, Vtp.v4 "
         "FROM Phases p, Reference r, Cp, Vtp "
         "WHERE p.index = r.index AND p.index = cp.index AND p.index = Vtp.index")
cursor = cnx.cursor()
cursor.execute(query)

In [ ]:
for name,formula,h,s,v,k0,k1,k2,k3,v1,v2,v3,v4 in cursor:
    print ("@interface " + name.title() + "Berman" + ": BermanProperties { }")
    print ("@end")
    print ()

In [ ]:
for name,formula,h,s,v,k0,k1,k2,k3,v1,v2,v3,v4 in cursor:
    modForm = formula.title().replace("(1)","").replace("(", "").replace(")","")
    print ("@implementation " + name.title() + "Berman")
    print ()
    print ("-(id)init {")
    print ("    if ((self = [super initWithH:" + str(h))
    print ("                               S:" + str(s))
    print ("                              k0:" + str(k0))
    print ("                              k1:" + str(k1))
    print ("                              k2:" + str(k2))
    print ("                              k3:" + str(k3))
    print ("                              v0:" + str(v))
    print ("                              v1:" + str(v3))
    print ("                              v2:" + str(v4))
    print ("                              v3:" + str(v1))
    print ("                              v4:" + str(v2) + "])) {")
    print ("    [self setPhaseFormula:@" + '"' + modForm + '"' + "];")
    print ("    [self setPhaseName:@" + '"' + name.title() + '"' + "];")
    print ("    }")
    print ("    return self;")
    print ("}")
    print ()
    print ("@end")
    print ()

Pandas dataframe generation code

In [ ]:
query = ("SELECT p.name AS Phase, p.formula AS Formula, p.short_abbrv AS sAbbrev, p.long_abbrv AS lAbbrev, "
         "r.h AS H_TrPr, r.s AS S_TrPr, r.v AS V_TrPr, "
         "Cp.k0, Cp.k1, Cp.k2, Cp.k3, Cp.k4, Cp.k5, Cp.k6, "
         "t.Tt AS T_lambda_Pr, T.Tr AS T_lambda_ref, t.l1, t.l2, t.Ht AS H_t, t.dTdP as k_lambda, "
         "Vtp.v3 as v1, Vtp.v4 as v2, Vtp.v1 as v3, Vtp.v2 as v4, "
         "d.d0 as d0, d.d1 as d1, d.d2 as d2, d.d4 as d3, d.d5 as d4, d.d6 as d5, d.Tmin AS T_D_ref, d.Tmax AS T_D "
         "FROM Phases p "
         "LEFT JOIN Reference r on p.index = r.index "
         "LEFT JOIN Cp on p.index = cp.index "
         "LEFT JOIN Vtp on p.index = Vtp.index "
         "LEFT JOIN Transition t ON p.index = t.index "
         "LEFT JOIN Disorder d ON p.index = d.index")
cursor = cnx.cursor()
cursor.execute(query)

In [ ]:
import pandas as pd
berman_df = pd.DataFrame(cursor.fetchall())
berman_df.columns = cursor.column_names

In [ ]:
print(berman_df.shape)

In [ ]:
berman_df.to_json('berman_1988.json')